In [32]:
library(dplyr)
library(ggplot2)
library(lattice)

Import "collisions_weather_data.csv" file

In [33]:
collision_data <- read.csv("https://raw.githubusercontent.com/uhi22018990/DataAnalytics/main/collisions_weather_data.csv", stringsAsFactors = TRUE)

Just as before, in the main document, let's have a quick look at our data:

In [34]:
head(collision_data)

,dim_date_year,dim_date_month,dim_date_day_number,collision_date,num_collisions,weather_date,temperature,dew_point,sea_level_pressure,visibility,wind_speed,max_sustained_wind_speed,max_wind_gust,max_temperature,min_temperature,total_precipitation,snow_depth,fog_occurred
,<int>,<int>,<int>,<fct>,<int>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>
1,2012,7,7,2012-07-01,538,2012-07-01,83.6,63.0,1008.9,9.7,4.1,9.9,18.1,93.0,66.0,0.00,999.9,0
2,2012,7,1,2012-07-02,564,2012-07-02,80.3,54.1,1011.6,10.0,3.8,15.0,999.9,88.0,66.9,0.00,999.9,0
3,2012,7,2,2012-07-03,664,2012-07-03,79.8,56.7,1012.8,10.0,2.9,12.0,999.9,88.0,63.0,0.00,999.9,0
4,2012,7,3,2012-07-04,432,2012-07-04,81.8,65.6,1009.1,9.2,3.6,11.1,999.9,91.0,68.0,0.06,999.9,0
5,2012,7,4,2012-07-05,591,2012-07-05,86.7,64.3,1007.4,9.4,3.8,15.0,999.9,93.9,70.0,99.99,999.9,0
6,2012,7,5,2012-07-06,638,2012-07-06,81.9,62.3,1011.7,10.0,3.1,9.9,999.9,91.0,66.9,0.00,999.9,0


In [35]:
tail(collision_data)

,dim_date_year,dim_date_month,dim_date_day_number,collision_date,num_collisions,weather_date,temperature,dew_point,sea_level_pressure,visibility,wind_speed,max_sustained_wind_speed,max_wind_gust,max_temperature,min_temperature,total_precipitation,snow_depth,fog_occurred
,<int>,<int>,<int>,<fct>,<int>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>
3764,2022,10,4,2022-10-20,275,2022-10-20,50.5,32.2,1014.3,10.0,5.0,8.9,15.9,57.0,42.1,0.00,999.9,0
3765,2022,10,5,2022-10-21,285,2022-10-21,53.8,34.3,1018.9,10.0,2.3,5.1,999.9,62.1,44.1,0.00,999.9,0
3766,2022,10,6,2022-10-22,278,2022-10-22,57.1,40.0,1023.7,10.0,2.4,6.0,999.9,66.0,46.0,0.00,999.9,0
3767,2022,10,7,2022-10-23,202,2022-10-23,57.5,46.7,1025.1,10.0,4.8,11.1,18.1,66.0,48.9,0.00,999.9,1
3768,2022,10,1,2022-10-24,216,2022-10-24,57.2,52.8,1021.8,6.0,6.9,9.9,20.0,61.0,54.0,0.16,999.9,1
3769,2022,10,2,2022-10-25,194,2022-10-25,64.0,62.2,1018.5,2.9,3.3,8.9,999.9,70.0,55.0,0.38,999.9,1


The data seems to be the same we have on big query


In [36]:
summary(collision_data)

 dim_date_year  dim_date_month   dim_date_day_number    collision_date
 Min.   :2012   Min.   : 1.000   Min.   :1.000       2012-07-01:   1  
 1st Qu.:2015   1st Qu.: 4.000   1st Qu.:2.000       2012-07-02:   1  
 Median :2017   Median : 7.000   Median :4.000       2012-07-03:   1  
 Mean   :2017   Mean   : 6.582   Mean   :3.999       2012-07-04:   1  
 3rd Qu.:2020   3rd Qu.:10.000   3rd Qu.:6.000       2012-07-05:   1  
 Max.   :2022   Max.   :12.000   Max.   :7.000       2012-07-06:   1  
                                                     (Other)   :3763  
 num_collisions       weather_date   temperature     dew_point     
 Min.   :  94.0   2012-07-01:   1   Min.   : 6.9   Min.   :-16.10  
 1st Qu.: 365.0   2012-07-02:   1   1st Qu.:42.6   1st Qu.: 27.30  
 Median : 549.0   2012-07-03:   1   Median :57.6   Median : 43.80  
 Mean   : 514.7   2012-07-04:   1   Mean   :56.4   Mean   : 42.04  
 3rd Qu.: 635.0   2012-07-05:   1   3rd Qu.:72.0   3rd Qu.: 58.30  
 Max.   :1161.0   2012-0

ah we have outliers here: 999.90 is indication of missing value

## normalization and export for DNN
let's create a dataset without outlier or null value. Exclude max wind gust and snow

In [37]:
collision_data_clean <- collision_data[ which(collision_data$visibility < 999),]
collision_data_clean <- collision_data_clean[ which(collision_data_clean$sea_level_pressure < 9999),]
collision_data_clean <- collision_data_clean[ which(collision_data_clean$wind_speed < 999),]
# collision_data_clean <- collision_data_clean[ which(collision_data_clean$max_wind_gust < 999),]
collision_data_clean <- collision_data_clean[ which(collision_data_clean$total_precipitation < 99),]
# collision_data_clean <- collision_data_clean[ which(collision_data_clean$snow_depth < 999),]
dim(collision_data_clean)
#head(collision_data_clean)
#tail(collision_data_clean)
#summary(collision_data_clean)
head(collision_data_clean)
ncol(collision_data_clean)


[1] 3521   18

,dim_date_year,dim_date_month,dim_date_day_number,collision_date,num_collisions,weather_date,temperature,dew_point,sea_level_pressure,visibility,wind_speed,max_sustained_wind_speed,max_wind_gust,max_temperature,min_temperature,total_precipitation,snow_depth,fog_occurred
,<int>,<int>,<int>,<fct>,<int>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>
1,2012,7,7,2012-07-01,538,2012-07-01,83.6,63.0,1008.9,9.7,4.1,9.9,18.1,93,66.0,0.00,999.9,0
2,2012,7,1,2012-07-02,564,2012-07-02,80.3,54.1,1011.6,10.0,3.8,15.0,999.9,88,66.9,0.00,999.9,0
3,2012,7,2,2012-07-03,664,2012-07-03,79.8,56.7,1012.8,10.0,2.9,12.0,999.9,88,63.0,0.00,999.9,0
4,2012,7,3,2012-07-04,432,2012-07-04,81.8,65.6,1009.1,9.2,3.6,11.1,999.9,91,68.0,0.06,999.9,0
6,2012,7,5,2012-07-06,638,2012-07-06,81.9,62.3,1011.7,10.0,3.1,9.9,999.9,91,66.9,0.00,999.9,0
7,2012,7,6,2012-07-07,490,2012-07-07,84.9,65.9,1010.1,9.5,3.1,14.0,17.1,95,70.0,0.00,999.9,0


[1] 18

Now remove the colum that are not interesting


In [38]:
#collision_data_clean <- collision_data_clean[,-1:-2] # remove dim_date_year, dim_date_month
collision_data_clean <- collision_data_clean[,-4] # remove collision_date
collision_data_clean <- collision_data_clean[,-5] #remove weather_date
collision_data_clean <- collision_data_clean[,-15:-16] #remove snow_depth,	fog_occurred
collision_data_clean <- collision_data_clean[,-11] # remove max_wind_gust
## then move num_collisions as first column
collision_data_clean <- collision_data_clean %>% relocate(num_collisions, .before = dim_date_year)
head(collision_data_clean)
summary(collision_data_clean)
sd(collision_data_clean$num_collisions)


,num_collisions,dim_date_year,dim_date_month,dim_date_day_number,temperature,dew_point,sea_level_pressure,visibility,wind_speed,max_sustained_wind_speed,max_temperature,min_temperature,total_precipitation
,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,538,2012,7,7,83.6,63.0,1008.9,9.7,4.1,9.9,93,66.0,0.00
2,564,2012,7,1,80.3,54.1,1011.6,10.0,3.8,15.0,88,66.9,0.00
3,664,2012,7,2,79.8,56.7,1012.8,10.0,2.9,12.0,88,63.0,0.00
4,432,2012,7,3,81.8,65.6,1009.1,9.2,3.6,11.1,91,68.0,0.06
6,638,2012,7,5,81.9,62.3,1011.7,10.0,3.1,9.9,91,66.9,0.00
7,490,2012,7,6,84.9,65.9,1010.1,9.5,3.1,14.0,95,70.0,0.00


 num_collisions  dim_date_year  dim_date_month   dim_date_day_number
 Min.   : 94.0   Min.   :2012   Min.   : 1.000   Min.   :1          
 1st Qu.:361.0   1st Qu.:2014   1st Qu.: 4.000   1st Qu.:2          
 Median :547.0   Median :2017   Median : 7.000   Median :4          
 Mean   :513.4   Mean   :2017   Mean   : 6.579   Mean   :4          
 3rd Qu.:633.0   3rd Qu.:2020   3rd Qu.: 9.000   3rd Qu.:6          
 Max.   :999.0   Max.   :2022   Max.   :12.000   Max.   :7          
  temperature      dew_point     sea_level_pressure   visibility    
 Min.   : 6.90   Min.   :-16.1   Min.   : 989       Min.   : 0.700  
 1st Qu.:43.30   1st Qu.: 27.8   1st Qu.:1012       1st Qu.: 8.500  
 Median :58.20   Median : 44.8   Median :1017       Median : 9.800  
 Mean   :57.04   Mean   : 42.6   Mean   :1017       Mean   : 9.009  
 3rd Qu.:72.50   3rd Qu.: 58.8   3rd Qu.:1022       3rd Qu.:10.000  
 Max.   :89.10   Max.   : 74.8   Max.   :1042       Max.   :10.000  
   wind_speed    max_sustained_win

[1] 160.8796

In [39]:
# wind_speed_numeric <- collision_data_clean[ , unlist(lapply(collision_data_clean, is.numeric))]
#wind_speed_numeric <- collision_data_clean[ , unlist(lapply(collision_data_clean, is.numeric))]
collision_data_clean[,1] <- scale(collision_data_clean[,1])
head(collision_data_clean)
summary(collision_data_clean)

## by scaling I'll have a Mean 0 for num_collision!

,num_collisions,dim_date_year,dim_date_month,dim_date_day_number,temperature,dew_point,sea_level_pressure,visibility,wind_speed,max_sustained_wind_speed,max_temperature,min_temperature,total_precipitation
,"<dbl[,1]>",<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,0.1529736,2012,7,7,83.6,63.0,1008.9,9.7,4.1,9.9,93,66.0,0.00
2,0.3145851,2012,7,1,80.3,54.1,1011.6,10.0,3.8,15.0,88,66.9,0.00
3,0.9361678,2012,7,2,79.8,56.7,1012.8,10.0,2.9,12.0,88,63.0,0.00
4,-0.5059040,2012,7,3,81.8,65.6,1009.1,9.2,3.6,11.1,91,68.0,0.06
6,0.7745563,2012,7,5,81.9,62.3,1011.7,10.0,3.1,9.9,91,66.9,0.00
7,-0.1453861,2012,7,6,84.9,65.9,1010.1,9.5,3.1,14.0,95,70.0,0.00


  num_collisions.V1   dim_date_year  dim_date_month   dim_date_day_number
 Min.   :-2.6068535   Min.   :2012   Min.   : 1.000   Min.   :1          
 1st Qu.:-0.9472277   1st Qu.:2014   1st Qu.: 4.000   1st Qu.:2          
 Median : 0.2089160   Median :2017   Median : 7.000   Median :4          
 Mean   : 0.0000000   Mean   :2017   Mean   : 6.579   Mean   :4          
 3rd Qu.: 0.7434771   3rd Qu.:2020   3rd Qu.: 9.000   3rd Qu.:6          
 Max.   : 3.0184697   Max.   :2022   Max.   :12.000   Max.   :7          
  temperature      dew_point     sea_level_pressure   visibility    
 Min.   : 6.90   Min.   :-16.1   Min.   : 989       Min.   : 0.700  
 1st Qu.:43.30   1st Qu.: 27.8   1st Qu.:1012       1st Qu.: 8.500  
 Median :58.20   Median : 44.8   Median :1017       Median : 9.800  
 Mean   :57.04   Mean   : 42.6   Mean   :1017       Mean   : 9.009  
 3rd Qu.:72.50   3rd Qu.: 58.8   3rd Qu.:1022       3rd Qu.:10.000  
 Max.   :89.10   Max.   : 74.8   Max.   :1042       Max.   :10.000  

In [40]:
# then export

write.csv(collision_data_clean, file="/content/collision_data_clean_dnn.csv")